In [1]:
import torch
import h5py
import numpy as np
import sys
import os
sys.path.append("../")
from agents.imitation.modules.networks import define_G
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
transfer_model = define_G(3, 3, 64, "resnet_9blocks", norm="instance", gpu_ids=[0])

if isinstance(transfer_model, torch.nn.DataParallel):
    transfer_model = transfer_model.module
    
if torch.cuda.is_available():
    transfer_model.cuda()

initialize network with normal


In [5]:
model_path = "../agents/imitation/model_transfer/non_croped/clear2rain_shift_1000_G_B_5.pth"

In [6]:
transfer_model.load_state_dict(torch.load(model_path, map_location='cuda:0'))
transfer_model.eval()

ResnetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): ResnetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Conv2d(256, 

In [7]:
data_= "/home/tai/ws/ijrr_2018/carla_cil_dataset/AgentHuman/chosen_weather_train/rainnoon_h5/data_05359.h5"
save_path = "/home/tai/ws/ijcai_2018/epochs_compare/5"
with h5py.File(data_, 'r') as f:
    img = np.array(f['rgb']).astype(np.float32)
    image_input = np.transpose(img, (0, 3, 1, 2))
    #plt.imshow(img*255)
    #print(img.dtype)
    
for i in range(200):
    with torch.no_grad():
        img_ts = torch.from_numpy(image_input[i:i+1]).cuda()
        img_out = transfer_model(img_ts)

    img_out = (img_out[0].cpu().numpy() + 1 )/ 2.0
    img_out = np.transpose(img_out, (1,2,0))*255
    #plt.imshow(img_out)
    Image.fromarray(img_out.astype(np.uint8)).save(os.path.join(save_path, str(i)+".png"))





